# AI Meeting Note Taker - Experiments

This notebook contains experiments and tests for various components.

## 1. Whisper Model Comparison

Compare different Whisper model sizes for accuracy vs speed tradeoff.

In [ ]:
from faster_whisper import WhisperModel
import time

# Test audio file
test_audio = "test_audio.wav"

models = ["tiny", "base", "small", "medium", "large-v2"]
results = []

for model_name in models:
    print(f"Testing {model_name}...")
    model = WhisperModel(model_name, device="cuda", compute_type="float16")
    
    start = time.time()
    segments, info = model.transcribe(test_audio)
    text = " ".join([s.text for s in segments])
    elapsed = time.time() - start
    
    results.append({
        "model": model_name,
        "time": elapsed,
        "words": len(text.split())
    })
    print(f"  Time: {elapsed:.2f}s, Words: {len(text.split())}")

## 2. Ollama Model Testing

Test different LLM models for summarization quality.

In [ ]:
import requests

def test_ollama(model, prompt):
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": model,
            "prompt": prompt,
            "stream": False
        }
    )
    return response.json()["response"]

# Test prompt
test_prompt = "Summarize this meeting transcript in 3 bullet points: ..."

models = ["llama3.1:8b", "mistral:7b", "phi3:mini"]
for model in models:
    print(f"\n=== {model} ===")
    result = test_ollama(model, test_prompt)
    print(result)

## 3. Audio Device Testing

List and test audio devices.

In [ ]:
import pyaudiowpatch as pyaudio

p = pyaudio.PyAudio()

print("=== All Audio Devices ===")
for i in range(p.get_device_count()):
    dev = p.get_device_info_by_index(i)
    loopback = dev.get("isLoopbackDevice", False)
    if loopback:
        print(f"[LOOPBACK] {i}: {dev['name']}")

print("\n=== Default Output ===")
default = p.get_default_output_device_info()
print(f"  {default['name']}")

## 4. Action Item Extraction Testing

Test the action item extraction logic with various formats.

In [ ]:
def extract_action_items(mom_content):
    """Extract action items from MoM content."""
    action_items = []
    in_action_section = False
    
    action_keywords = [
        'action item', 'next step', 'follow-up',
        'practice exercise', 'key takeaway'
    ]
    
    for line in mom_content.split('\n'):
        line_lower = line.lower()
        
        if line.startswith('#') and any(kw in line_lower for kw in action_keywords):
            in_action_section = True
            continue
        
        if in_action_section and line.startswith('#'):
            if not any(kw in line_lower for kw in action_keywords):
                in_action_section = False
            continue
        
        if in_action_section:
            stripped = line.strip()
            if stripped.startswith('|') and '---' not in stripped:
                cells = [c.strip() for c in stripped.split('|')]
                cells = [c for c in cells if c]
                if cells and cells[0].isdigit():
                    action_items.append(cells[1:])
            elif stripped.startswith('- '):
                action_items.append([stripped[2:]])
    
    return action_items

# Test
test_mom = """
## Action Items
| # | Action Item | Owner | Deadline |
|---|-------------|-------|----------|
| 1 | Review proposal | John | Dec 20 |
| 2 | Send report | Jane | Dec 25 |

## Next Section
"""

items = extract_action_items(test_mom)
print(f"Found {len(items)} items:")
for item in items:
    print(f"  - {item}")